In [82]:
import numpy as np
from scipy.optimize import fmin

In [83]:
heston_parameters = [0.02, 1.5, 0.05, 0.18, 0.5, 0.04]

# Simulation of stock prices

In [84]:
def simulate_heston(params, S0, T):
    dt = 1.0/T
    mu, kappa, theta, lbd, rho, v0 = params
    sim_array = np.zeros((T+1,2))
    sim_array[0,:] = S0, v0
    w = np.random.normal(size=(T,2))
    for i in range(1,T+1):
        prev_s, prev_v = sim_array[i-1, :]
        vi = prev_v + kappa*(theta-max(0,prev_v))*dt + lbd*np.sqrt(max(0,prev_v)*dt)*w[i-1][0]
        w_corr = rho * w[i-1, 0] + np.sqrt(1-rho**2) * w[i-1, 1]
        si = prev_s + (mu - 0.5*max(0,prev_v))*dt + np.sqrt(max(0,prev_v)*dt)*w_corr
        sim_array[i, :] = si,vi
    return sim_array[:,0]

# Extended Kalman Filter

In [95]:
def extended_kalman(params, H, P, T, s):
    mu, kappa, theta, lbd, rho, v0 = params
    dt = 1.0/T
    F = np.array([[1., -0.5*dt], 
                  [0., 1 - kappa*dt]])
    Q = np.array([[1., rho], 
                 [rho, 1.]])
    loglikelihood = 0
    v = v0
    
    for t in range(1, T+1):
        U = np.diag([np.sqrt(v*dt), lbd * np.sqrt(v*dt)])
        prev_y = s[t-1]
        x1 = np.array([prev_y + (mu - 0.5*v)*dt,
                      v + kappa*(theta-v)*dt])
        P1 = F@P@F.T + U@Q@U.T
        S = H@P1@H.T
        K = P1@H.T/S
        v1 = x1[1]
        e = s[t] - s[t-1] - (mu - 0.5*v1)*dt
        loglikelihood += np.log(S) + e**2/S
        x = x1 + K*e
        v = x[1]
        P = (np.diag([1., 1.]) - np.outer(K, H)) @ P1
    return loglikelihood

In [96]:
H = np.array([1, 0])
P = np.diag([0.01, 0.01])
initial_param = [0.033, 1.7, 0.08, 0.14, 0.36, 0.023]
T = 10*365
S0 = 3.66
s = simulate_heston(heston_parameters, S0, T)
ekm = fmin(extended_kalman, initial_param, args=(H,P,T,s), full_output=True, disp=True, xtol=10, ftol=20)
ekm_param = np.round(ekm[0], 4)

Optimization terminated successfully.
         Current function value: -37200.594403
         Iterations: 23
         Function evaluations: 38


In [98]:
print("Est. Parameters: ", ekm_param)
print("True Parameters: ", heston_parameters)
print("Absolute Error:  ", np.max(ekm_param - heston_parameters))
print("RMSE:            ", np.linalg.norm(ekm_param - heston_parameters))

Est. Parameters:  [0.0269 2.5144 0.0514 0.1345 0.2493 0.0355]
True Parameters:  [0.02, 1.5, 0.05, 0.18, 0.5, 0.04]
Absolute Error:   1.0144000000000002
RMSE:             1.045943554882385


# Particle Filter

## Sampling Importance Resampling

In [70]:
# Avoids degeneracy problem
def resampling(w,x):
    N = len(x)
    c = np.cumsum(w)
    U = (np.arange(N) + np.random.rand(N))/N
    i = 1
    x_new = np.zeros(N)
    for j in range(N):
        while U[j] > c[i]:
            i += 1
        x_new[j] = x[i]
    w_new = np.ones(N)/N
    return x_new, w_new

In [72]:
def n(x,m,s):
    return np.exp(-((x-m)**2)/(2*s**2))/(np.sqrt(2*np.pi)*s)

In [79]:
def particleFilter(params, N, T, s):
    mu, kappa, theta, lbd, rho, v0 = params
    dt = 1.0/T
    w = np.ones(N)/N
    c = 1/(1 + (rho*lbd*dt)/2)
    c2 = rho*lbd*dt
    v = v0*np.ones(N)
    loglikelihood = 0
    for t in range(1, T+1):
        z1 = np.random.normal(N)
        z2 = np.random.normal(N)
        vt = (v + kappa*(theta-v)*dt + (-c2/2)*v \
              + lbd*np.sqrt(v*(1-rho**2)*dt)*z2 \
              + rho*lbd*np.sqrt(v*dt)*z1)/c
        yt, y_prev = s[t], s[t-1]
        m_i = v + kappa*(theta - v)*dt + lbd*rho*(yt-y_prev - (mu - 0.5*v)*dt)
        s_i = lbd*np.sqrt(v*(1-rho**2)*dt)
        m_T = c*(v + kappa*(theta - v)*dt + 0.5*lbd*rho*v*dt)
        s_T = c*lbd*np.sqrt(v*dt)
        m_L = y_prev + (mu - 0.5*vt) * dt
        s_L = np.sqrt(v * dt)
        r = n(yt, m_L, s_L)*n(vt, m_T, s_T) / n(vt, m_i, s_i)
        w *= r
        loglikelihood += np.log(np.sum(w))
        w /= np.sum(w)
        v, w = resampling(w, vt)
    return -loglikelihood

In [80]:
initial_param = [0.033, 1.7, 0.08, 0.14, 0.36, 0.023]
T = 10*365
S0 = 3.66
s = simulate_heston(heston_parameters, S0, T)
N = 20
pf = fmin(particleFilter, initial_param, args=(N,T,s), full_output=True, disp=True, xtol=10, ftol=20)
pf_param = np.round(pf[0], 4)

/var/folders/m4/52jcfz5d7tldnd1yw00622fm0000gn/T/ipykernel_16277/1032910003.py:22: RuntimeWarning: divide by zero encountered in true_divide
  r = n(yt, m_L, s_L)*n(vt, m_T, s_T) / n(vt, m_i, s_i)
/var/folders/m4/52jcfz5d7tldnd1yw00622fm0000gn/T/ipykernel_16277/1032910003.py:25: RuntimeWarning: invalid value encountered in true_divide
  w /= np.sum(w)


In [81]:
print("Est. Parameters: ", pf_param)
print("True Parameters: ", heston_parameters)
print("Absolute Error:  ", np.max(pf_param - heston_parameters))
print("RMSE:            ", np.linalg.norm(pf_param - heston_parameters))

Est. Parameters:  [0.0385 1.3094 0.0804 0.0622 0.6373 0.0271]
True Parameters:  [0.02, 1.5, 0.05, 0.18, 0.5, 0.04]
Absolute Error:   0.13729999999999998
RMSE:             0.26549822974927734
